# Output Parsing

This notebook covers functionality related to output parsing. For more information, see:

- [List of all Output Parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)

- [OpenAI Function Calling](https://python.langchain.com/docs/modules/model_io/output_parsers/types/openai_functions)

## Converting Messages

In [2]:
import logging
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [3]:
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

load_dotenv()

True

In [4]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
model = ChatOpenAI()

In [5]:
chain = prompt | model

In [6]:
chain.invoke({"topic": "bears"})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="Why don't bears wear socks? \n\nBecause they have bear feet!")

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = prompt | model | parser

In [8]:
chain.invoke({"topic": "bears"})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


"Why don't bears wear shoes?\n\nBecause they already have bear feet!"

## OpenAI Function Calling

In [9]:
from langchain.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [10]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


openai_functions = [convert_pydantic_to_openai_function(Joke)]

In [11]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [12]:
parser = JsonOutputFunctionsParser()

In [13]:
chain = prompt | model.bind(functions=openai_functions) | parser

In [15]:
chain.invoke({"topic": "bears"})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'setup': "Why don't bears wear shoes?",
 'punchline': 'Because they have bear feet!'}

## Streaming

In [16]:
for s in chain.stream({"topic": "bars"}):
    print(s)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why don'}
{'setup': "Why don't"}
{'setup': "Why don't scientists"}
{'setup': "Why don't scientists trust"}
{'setup': "Why don't scientists trust atoms"}
{'setup': "Why don't scientists trust atoms?"}
{'setup': "Why don't scientists trust atoms?", 'punchline': ''}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything'}
{'setup': "Why don't scientists trust atoms?", 'punchline': 'Because they make up everything!'}
